# Visualisation of LFP & Sleep scoring

### Import recordings

Load packages

In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path
from scipy import signal
import os
from IPython.display import display
from ipyfilechooser import FileChooser
from ephyviewer import mkQApp, MainViewer, TraceViewer
import ephyviewer
from scipy.stats import zscore
from itertools import groupby
from ephyviewer import mkQApp, MainViewer, TraceViewer, TimeFreqViewer
from ephyviewer import InMemoryAnalogSignalSource
from ephyviewer import mkQApp, MainViewer, TraceViewer, CsvEpochSource, EpochEncoder
import json
import IPython
from matplotlib import cm
from matplotlib.colors import to_hex
import ast
from ephyviewer import AnalogSignalSourceWithScatter
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.colors as mcolors
from scipy import stats
import re
import xml.etree.ElementTree as ET
from datetime import datetime, timedelta
from datetime import datetime, timezone
from zoneinfo import ZoneInfo  # Only in Python 3.9+
import shutil

%matplotlib widget

In [ ]:
#Load LFP coordinates 
notebook_path = Path("/".join(IPython.extract_module_locals()[1]["__vsc_ipynb_file__"].split("/")[-5:]))
Channels = f'{notebook_path.parent}/_LFP_coordinates_of_all_mice.csv'
all_LFPcoordinates = pd.read_csv(Channels, index_col=0)

Choose OpenEphys folder

In [ ]:
try: # tries to retrieve dpath either from a previous run or from a previous notebook
    %store -r dpath
    #dpath=dpath.parent
except:
    print("the path was not defined in store")
dpath = "//10.69.168.1/crnldata/forgetting/"

fc1 = FileChooser(dpath,select_default=True, show_only_dirs = True, title = "<b>Go inside the folder containing the LFP raw file</b>")
display(fc1)

# Sample callback function
def update_my_folder(chooser):
    global dpath
    dpath = chooser.selected
    %store dpath
    return 

# Register callback function
fc1.register_callback(update_my_folder)

#### Load LFPs data 

In [ ]:
folder_base = Path(dpath) 

if Path(f'{folder_base}/DataFrame_rawdataDS.pkl').exists(): # prefer loading downsample file over original file
    print('DataFrame_rawdataDS.pkl file')
    LFPfile = Path(f'{folder_base}/DataFrame_rawdataDS.pkl')
    LFPs_df = pd.read_pickle(LFPfile)
    samplerate = 1000 
    numchannel = LFPs_df.shape[1]
    rec_ch_list = LFPs_df.columns.values
    # Load LFPs timestamps 
    for file_pathTS in folder_base.parent.parent.glob('**/continuous/*/timeStampsDS.npy'):
        print('LFPs timestamps file found')
        LFPtimestamps = np.load(file_pathTS)  
elif Path(f'{folder_base}/RawDataChannelExtractedDS.npy').exists():
    print('RawDataChannelExtractedDS.npy file')
    LFPfile = Path(f'{folder_base}/RawDataChannelExtractedDS.npy')
    DataRec = np.load(LFPfile, mmap_mode= 'r')
    samplerate = 1000 
    numchannel = DataRec.shape[1]
    rec_ch_list = np.arange(0,numchannel,1)
    LFPs_df=pd.DataFrame(DataRec, columns=rec_ch_list)
    print('No metadata found')
elif Path(f'{folder_base}/continuous.dat').exists():
    print('continuous.dat file')
    LFPfile = Path(f'{folder_base}/continuous.dat')
    DataRec = np.memmap(LFPfile, dtype="int16", mode='r') 
    #DataRec2 = np.fromfile(LFPfile, dtype="int16")  
    filepath = Path(os.path.join(folder_base.parent.parent, f'structure.oebin'))
    #filepath = Path(os.path.join(folder_base, f'structure.oebin'))
    with open(filepath) as f:
        metadata = json.load(f)
    samplerate = metadata['continuous'][0]['sample_rate']  
    numchannel = metadata['continuous'][0]['num_channels'] 
    rec_ch_list = np.array([int(''.join(c for c in metadata['continuous'][0]['channels'][x]['channel_name'] if c.isdigit()))-1 for x in range(0, len(metadata['continuous'][0]['channels']))])
    DataRec = DataRec.reshape(-1,numchannel)
    print('Metadata found')
    LFPs_df=pd.DataFrame(DataRec, columns=rec_ch_list) 
    # Load LFPs timestamps 
    try: 
        for file_pathTS in folder_base.parent.parent.glob('**/continuous/*/timeStamps.npy'):
            print('LFPs timestamps file found')
            LFPtimestamps = np.load(file_pathTS) 
    except: 
        None
else: 
    print('no LFPs file found')

print('sample rate =', samplerate, 'Hz')
print(numchannel, 'channels recorded')
print(round(LFPs_df.shape[0]/samplerate/60), 'min of recording')

Downsample LFP data to 1kHz if needed & save it

In [ ]:
if samplerate > 1000:
    new_sampling_rate = 1000 # Hz
    Nmber_points = int(np.shape(LFPs_df)[0] * new_sampling_rate / samplerate)
    LFPs_df_DS = pd.DataFrame(signal.resample(LFPs_df, Nmber_points, axis = 0), columns=LFPs_df.columns.values)
    samplerate = new_sampling_rate
    LFPs_df_DS.to_pickle(f'{LFPfile.parent}/DataFrame_rawdataDS.pkl')
    LFPs_df = LFPs_df_DS
    LFPtimestampsDS = LFPtimestamps[::int(samplerate/new_sampling_rate)][:-1]
    np.save(f'{file_pathTS.parent}/timeStampsDS.npy', LFPtimestampsDS)
    LFPtimestamps = LFPtimestampsDS
# eventually delete original files manually to gain space

Identify mouse

In [ ]:
mouse = []
pos_mice = []
for mouse_name in all_LFPcoordinates.index:
    if mouse_name in LFPfile.__str__():
        mouse.append(mouse_name)
        pos_mice.append(LFPfile.__str__().find(mouse_name)) 
mouse = [x for _, x in sorted(zip(pos_mice, mouse))] # sort mouse in the same order as they appear in the path

if len(mouse) > 1: # found multiple mouse name in the path
    if max(rec_ch_list) <= 32: # no channels superior to 32, so only one mouse recorded
        id = 1 # change to 0 to see the first mouse name found, 1 the second, etc
        ID = 0
        print(f"/!/ Mutliple mice name found in the path but only mouse recorded = {mouse}. The n°{id+1} was choosen automatically = {mouse[id]}.")
        mouse = mouse[id]
    else:
        ###################################################################################################################################
        ID = 1 # choose 0 to see the first mouse recorded, 1 the second, 2 the third, 3 the fourth (only 4 mice can be recorded at the same time)
        ###################################################################################################################################
        print(f"/!/ Mutliple mice recorded at the same time = {mouse}. The n°{ID+1} was choosen automatically = {mouse[ID]}.")
        mouse = mouse[ID] 
elif max(rec_ch_list) > 32 & len(mouse) == 1:
    ###################################################################################################################################
    ID = 0 # choose 0 to see the first mouse recorded, 1 the second, 2 the third, 3 the fourth (only 4 mice can be recorded at the same time)
    ###################################################################################################################################
    mouse = mouse[0]
    print(f"/!/ Mutliple mice recorded at the same time, but only one found in the path = {mouse}. The n°{ID+1} was choosen automatically = {mouse}.")
elif len(mouse) == 1: # found only one mouse name in the path
    #ID = 0
    ID = int(rec_ch_list[0]/32)
    mouse = mouse[0]
    print(f'One mouse recorded only = {mouse}, on the OE input n°{ID+1}')

Identify electrodes & create differential LFPs

In [ ]:
all_LFPcoordinates = all_LFPcoordinates.astype(str)
for region in all_LFPcoordinates.loc[mouse].index:
    locals()[region] = []
    locals()[f'{region}_ch'] = []

RecordedArea = []
ChoosenChannels = []
combined = []
if mouse:
    rec_ch_list_mouse = [value for value in rec_ch_list if 0+(ID*32) <= value <= 31+(ID*32)]
    for rec_ch in rec_ch_list_mouse:
        for idx, LFPcoord_str in enumerate(all_LFPcoordinates.loc[mouse]):
            region = all_LFPcoordinates.loc[mouse].index[idx]
            if LFPcoord_str != 'nan':
                LFPcoord = LFPcoord_str.split('_')[:2] # only take into account the 2 first of electrode of that region                 
                num_ch = np.where(str(rec_ch-(ID*32)) == np.array(LFPcoord))[0]
                if len(num_ch) > 0:
                    region = all_LFPcoordinates.loc[mouse].index[idx]
                    LFP = locals()[region]
                    LFP = LFP-np.array(LFPs_df[(rec_ch)]) if len(LFP) > 0 else np.array(LFPs_df[(rec_ch)])
                    locals()[region] = LFP
                    locals()[f'{region}_ch'].append(rec_ch)
                    break
                continue
    
    for region in all_LFPcoordinates.loc[mouse].index:
        LFP = locals()[region]
        LFP_ch = locals()[f'{region}_ch']
        if len(LFP) > 0:
            combined = zscore(LFP[:,np.newaxis]) if len(combined) == 0 else np.append(combined, zscore(LFP[:,np.newaxis]), axis=1)
            RecordedArea.append(region) 
            ChoosenChannels.append(LFP_ch) 
else:
    print("/!/ No mouse name found in the path OR in the csv file '_LFP_coordinates_of_all_mice.csv'")
    mouse = '' # fill mouse name
    RecordedArea = ['EMG','S1','PFC','CA1'] 
    EMG = LFPs_df[0]
    S1 = LFPs_df[0]-LFPs_df[1]
    combined = np.stack([zscore(EMG), zscore(S1)], axis=1)

print(mouse)
print(RecordedArea)
print(ChoosenChannels) 

Create OpenEphys folder if needed

In [ ]:
if LFPfile.name != 'DataFrame_rawdataDS.pkl':

    for file_path in folder_base.parents[1].rglob('sync_messages.txt'):
        with open(file_path, "r") as f:
            lines = f.readlines()
    timestamp_ms = int(re.sub(r'\D', '', lines[0].split(': ')[1]))
    timestamp_s = timestamp_ms / 1000 # if Original timestamp in milliseconds
    utc_dt = datetime.fromtimestamp(timestamp_s, tz=timezone.utc)
    start_time = utc_dt.astimezone(ZoneInfo("Europe/Paris")).strftime("%H-%M-%S")

    time_obj = datetime.strptime(start_time, "%H-%M-%S")
    minutes_to_add = LFPs_df.shape[0]/samplerate/60
    delta = timedelta(minutes=minutes_to_add)
    new_time = time_obj + delta
    new_time_str = new_time.strftime("%H-%M-%S")
    
    finaldir=Path(f'{folder_base.parents[5]}/{start_time}_{new_time_str}/')
    os.makedirs(Path(f'{finaldir}/OpenEphys/'), exist_ok=True)   
     
    # Save LFP timestamps
    np.save(f'{finaldir}/OpenEphys/LFP_timeStamps.npy', LFPtimestamps)
    # Save TTL events timestamps
    for TTLfile in folder_base.parents[1].rglob('TTL/timestamps.npy'):
        TTLtimestamps = np.load(TTLfile) 
        np.save(f'{finaldir}/OpenEphys/TTL_timeStamps.npy', TTLtimestamps)
    # Save setting file
    for settingfile in folder_base.parents[3].rglob('settings.xml'):
        shutil.copy2(settingfile, f'{finaldir}/OpenEphys/settings.xml')
    # Save metadata file
    for metadatafile in folder_base.parents[1].rglob('structure.oebin'):
        shutil.copy2(metadatafile, f'{finaldir}/OpenEphys/structure.oebin')
    # Save sync file
    for metadatafile in folder_base.parents[1].rglob('sync_messages.txt'):
        shutil.copy2(metadatafile, f'{finaldir}/OpenEphys/sync_messages.txt')
    print(f"OE data saved under {finaldir}")
else :
    finaldir= Path(LFPfile).parents[1]
    print(f"OpenEphys folder already completed")

Move associated Miniscope folders into OpenEphys folder 

In [ ]:
def move_folder_if_in_range(folder_a, folder_b):
    # Extract the time from folder A's name
    time_a = datetime.strptime(os.path.basename(folder_a), "%H_%M_%S")
    # Extract the start and end times from folder B's name
    base_b = os.path.basename(folder_b)
    start_str, end_str = base_b.split('_')
    start_time = datetime.strptime(start_str, "%H-%M-%S")
    end_time = datetime.strptime(end_str, "%H-%M-%S")
    
    # Check if A's time is within B's time range
    if start_time <= time_a <= end_time:
        # Move folder A into folder B
        dest = os.path.join(folder_b, os.path.basename(folder_a))
        shutil.move(folder_a, dest)
        print(f"Moved {folder_a.name} into {folder_b.name}")
    #else:
    #    print(f"{folder_a.name} not within range of {folder_b.name}")

v4_miniscope_folders = list(finaldir.parent.glob('*/*V4_Miniscope*/'))
for V4folder in v4_miniscope_folders:
    time_folder = V4folder.parent  
    #print(f"Checking Miniscope folder: {time_folder}")
    move_folder_if_in_range(time_folder, finaldir)

#### Score sleep manually


In [ ]:
StagesNb = 6 # 4 stages : Wake, NREM, REM, undefined OR 6 stages : AW (Active wake), QW (Quiet wake), NREM, IS (Intermediate sleep), REM, undefined 

labels = ['AW_1', 'QW_2', 'NREM_3', 'IS_4',  'REM_5', 'undefined_6'] if StagesNb == 6 else ['Wake_1', 'NREM_2', 'REM_3', 'undefined_4'] 
epoch_dur = 5 # define epoch duration in sec
winlen = 20 # default window length in sec

SleepScoring_filename = f'{finaldir}/OpenEphys/EphyViewer_ManualScor{epoch_dur}s_{len(labels)}Stages.csv'
#SleepScoring_filename = f'{dpath}/Sleep_Scoring_{len(labels)}Stages_{epoch_dur}sEpoch.csv'
source_epoch = CsvEpochSource(SleepScoring_filename, labels)

#you must first create a main Qt application (for event loop)
app = mkQApp()

t_start = 0.

#Create the main window that can contain several viewers
win = MainViewer(debug=False, show_auto_scale=True)

#create a viewer for signal
source =InMemoryAnalogSignalSource(combined, samplerate, t_start, channel_names=RecordedArea)
view1 = TraceViewer(source=source)

view1.params['xsize']= winlen
view1.params['display_labels'] = True
view1.params['scale_mode'] = 'same_for_all'
colormap = np.insert(['#88FF88', '#8888FF', '#FF8888']* 10, 0, '#FFFFFF')
for idx, ch in enumerate(RecordedArea): 
    view1.by_channel_params[f'ch{idx}', 'color'] = colormap[idx] #FF0000 red, #00FF00 green, and #0000FF blue
view1.auto_scale()

#create a viewer for the encoder itself
view2 = EpochEncoder(source=source_epoch, name='Sleep Scoring')

view2.params['xsize'] = winlen
view2.params['new_epoch_step'] = epoch_dur

if StagesNb == 6:
    view2.by_label_params['label0', 'color'] = '#ffcd69' #AW
    view2.by_label_params['label1', 'color'] = '#69ffe6' #QW
    view2.by_label_params['label2', 'color'] = '#69cfff' #NREM
    view2.by_label_params['label3', 'color'] = '#cd69ff' #IS
    view2.by_label_params['label4', 'color'] = '#ff69cd' #REM
    view2.by_label_params['label5', 'color'] = '#8f8f8f' #undefined
else: 
    view2.by_label_params['label0', 'color'] = '#ffcd69' #AW
    view2.by_label_params['label1', 'color'] = '#69cfff' #NREM
    view2.by_label_params['label2', 'color'] = '#ff69cd' #REM
    view2.by_label_params['label3', 'color'] = '#8f8f8f' #undefined


view2.controls.hide()

# FFT
view3 = TimeFreqViewer(source=source, name='FFT')

view3.params['show_axis'] = True
view3.params['timefreq', 'f_start'] = 1
view3.params['timefreq', 'f_stop'] = 30
view3.params['timefreq', 'deltafreq'] = 1 #interval in Hz
view3.params['xsize'] = winlen
for idx, ch in enumerate(RecordedArea):
    if ch == 'EMG': 
        view3.by_channel_params[f'ch{idx}', 'visible'] = False
    else:        
        view3.by_channel_params[f'ch{idx}', 'clim'] = 1
        view3.by_channel_params[f'ch{idx}', 'visible'] = True

#show main window and run Qapp

win.add_view(view1)
win.add_view(view3)
win.add_view(view2)
win.navigation_toolbar.spinbox_xsize.setValue(winlen)
win.show()

app.exec()

# press '1', '2', '3', '4' etc, to encode state.
# or toggle 'Time range selector' and then use 'Insert within range'

#### Save signals of this mouse only 

In [ ]:
if samplerate <= 1000 and LFPfile.name != 'DataFrame_rawdataDS.pkl':
    LFPs_df_mouse=LFPs_df[rec_ch_list_mouse]
    LFPs_df_mouse = LFPs_df_mouse.copy()
    LFPs_df_mouse.rename(columns={x: x - (ID * 32) for x in rec_ch_list_mouse}, inplace=True)
    file_name = 'DataFrame_rawdataDS.pkl'
    file_path = os.path.join(Path(f'{finaldir}/OpenEphys/'), file_name)
    LFPs_df_mouse.to_pickle(file_path)
elif samplerate > 1000 and LFPfile.name != 'DataFrame_rawdataDS.pkl': # best to downsample too
    LFPs_df_mouse=LFPs_df[rec_ch_list_mouse]
    LFPs_df_mouse = LFPs_df_mouse.copy()
    LFPs_df_mouse.rename(columns={x: x - (ID * 32) for x in rec_ch_list_mouse}, inplace=True)
    new_sampling_rate = 1000 # Hz
    Nmber_points = int(np.shape(LFPs_df_mouse)[0] * new_sampling_rate / samplerate)
    LFPs_df_mouse_DS = pd.DataFrame(signal.resample(LFPs_df_mouse, Nmber_points, axis = 0), columns=LFPs_df_mouse.columns.values)
    samplerate = new_sampling_rate
    file_name = 'DataFrame_rawdataDS.pkl'
    file_path = os.path.join(Path(f'{finaldir}/OpenEphys/'), file_name)
    LFPs_df_mouse_DS.to_pickle(file_path)
    LFPs_df_mouse = LFPs_df_mouse_DS
else: 
    print("No need to save in this format, it was already done")

### AccuSleePy

Save data for automated scoring

In [ ]:
if samplerate == 1000: # Save "eeg" like lfp & emg in the good format for AccuSleep
    if EnthC.any() & EMG.any(): 
        data = pd.DataFrame({'eeg': EnthC, 'emg': EMG})
        eeg='EnthC'
    elif CA1.any() & EMG.any(): 
        data = pd.DataFrame({'eeg': CA1, 'emg': EMG})
        eeg='CA1'
    elif RH.any() & EMG.any(): 
        data = pd.DataFrame({'eeg': RH, 'emg': EMG})
        eeg='RH'
    print(f'{eeg} and EMG found')
    #data.to_parquet(f"{dpath}/EegEmg_{mouse.replace(' ', '')}.parquet", index=False)   
    file_name = f'AccuSleep_{eeg}EMG.parquet'
    file_path = os.path.join(Path(f'{finaldir}/OpenEphys/'), file_name)
    data.to_parquet(file_path, index=False) 
    print('eeg & emg saved')
else: 
    print('change the sampling rate to 1000 Hz to save the data in parquet format')

try: # Save sleep scoring in the good format for AccuSleep
    SleepScoring = pd.read_csv(SleepScoring_filename)
    rounded_duration = SleepScoring['duration'].round().astype(int) # in case the duration is not an integer, round it to the nearest integer

    Accusleep_SleepScoring = pd.DataFrame({
        'brain_state': SleepScoring.loc[SleepScoring.index.repeat(rounded_duration), 'label'].values
    }).reset_index(drop=True).replace(r'\D', '', regex=True).replace('4', '-1', regex=True) if len(labels) == 4 else pd.DataFrame({
        'brain_state': SleepScoring.loc[SleepScoring.index.repeat(rounded_duration), 'label'].values
    }).reset_index(drop=True).replace(r'\D', '', regex=True).replace('6', '-1', regex=True)

    target_length = int(len(EMG)/samplerate)
    current_length = len(Accusleep_SleepScoring)
    if current_length < target_length:
        filler = pd.DataFrame(-1, index=range(target_length - current_length), columns=Accusleep_SleepScoring.columns) # fill with -1 == undefined state if sleep scoring is shorter than the length of the EMG signal
        Accusleep_SleepScoring = pd.concat([Accusleep_SleepScoring, filler], ignore_index=True)
    elif len(Accusleep_SleepScoring) > target_length:
        Accusleep_SleepScoring = Accusleep_SleepScoring.iloc[:target_length] # truncate if sleep scoring is longer than the length of the EMG signal

    epoch_duration = 5 # in seconds    
    Accusleep_SleepScoring=Accusleep_SleepScoring.iloc[::epoch_duration]

    file_name = f'AccuSleep_ManualScor{epoch_duration}.csv'
    file_path = os.path.join(Path(f'{finaldir}/OpenEphys/'), file_name)
    Accusleep_SleepScoring.to_csv(file_path, index=False)
    print('Sleep scoring saved')
except:
    print('No manual sleep scoring found')

Check AccuSleePy automatic scoring 

In [ ]:
SleepScoring_filename = Path(f'{finaldir}/OpenEphys/AccuSleep_AutoScor5.csv')
SleepScoring = pd.read_csv(SleepScoring_filename)
SleepScoring.rename(columns={'brain_state': 'label'}, inplace=True)
SleepScoring=SleepScoring.drop('confidence_score', axis=1)
epoch_dur = int([int(num) for num in re.findall(r'\d+', SleepScoring_filename.name)][0])
SleepScoring['duration']=epoch_dur
SleepScoring['time'] = list(range(0, len(SleepScoring) * epoch_dur, epoch_dur))
SleepScoring['label'] = SleepScoring['label'].replace({1: 'Wake_1', 2: 'NREM_2', 3: 'REM_3', -1: 'undefined_4'})
AutoSleepScoring_filename = os.path.join(Path(f'{finaldir}/OpenEphys/'), f'EphyViewer_AutoScor{epoch_duration}.csv')
SleepScoring.to_csv(AutoSleepScoring_filename, index=False)

In [ ]:
StagesNb = 6 # 4 stages : Wake, NREM, REM, undefined OR 6 stages : AW (Active wake), QW (Quiet wake), NREM, IS (Intermediate sleep), REM, undefined 
labels = ['AW_1', 'QW_2', 'NREM_3', 'IS_4',  'REM_5', 'undefined_6'] if StagesNb == 6 else ['Wake_1', 'NREM_2', 'REM_3', 'undefined_4'] 

winlen = 20 # default window length in sec
source_epoch = CsvEpochSource(AutoSleepScoring_filename, labels)

#you must first create a main Qt application (for event loop)
app = mkQApp()

t_start = 0.

#Create the main window that can contain several viewers
win = MainViewer(debug=False, show_auto_scale=True)

#create a viewer for signal
source =InMemoryAnalogSignalSource(combined, samplerate, t_start, channel_names=RecordedArea)
view1 = TraceViewer(source=source)

view1.params['xsize']= winlen
view1.params['display_labels'] = True
view1.params['scale_mode'] = 'same_for_all'
colormap = np.insert(['#88FF88', '#8888FF', '#FF8888']* 10, 0, '#FFFFFF')
for idx, ch in enumerate(RecordedArea): 
    view1.by_channel_params[f'ch{idx}', 'color'] = colormap[idx] #FF0000 red, #00FF00 green, and #0000FF blue
view1.auto_scale()

#create a viewer for the encoder itself
view2 = EpochEncoder(source=source_epoch, name='Sleep Scoring')
view2.params['xsize'] = winlen
view2.params['new_epoch_step'] = epoch_dur

view2.by_label_params['label0', 'color'] = '#ffcd69' #AW
view2.by_label_params['label1', 'color'] = '#69cfff' #NREM
view2.by_label_params['label2', 'color'] = '#ff69cd' #REM
view2.by_label_params['label3', 'color'] = '#8f8f8f' #undefined
view2.controls.hide()

# FFT
view3 = TimeFreqViewer(source=source, name='FFT')

view3.params['show_axis'] = True
view3.params['timefreq', 'f_start'] = 1
view3.params['timefreq', 'f_stop'] = 20
view3.params['timefreq', 'deltafreq'] = 1 #interval in Hz
view3.params['xsize'] = winlen
#addItem.ctrl.logXCheck.isChecked()

for idx, ch in enumerate(RecordedArea):
    if ch == 'EMG': 
        view3.by_channel_params[f'ch{idx}', 'visible'] = False
    else:        
        view3.by_channel_params[f'ch{idx}', 'clim'] = 1
        view3.by_channel_params[f'ch{idx}', 'visible'] = True


#show main window and run Qapp

win.add_view(view1)
win.add_view(view3)
win.add_view(view2)
win.navigation_toolbar.spinbox_xsize.setValue(winlen)
win.show()

app.exec()

# press '1', '2', '3', '4' etc, to encode state.
# or toggle 'Time range selector' and then use 'Insert within range'

### Check all channels

In [ ]:
app = mkQApp()
winlen = 20 # default window length in sec
t_start = 0.

#Create the main window that can contain several viewers
win = MainViewer(debug=False, show_auto_scale=True)
view1 = TraceViewer.from_numpy(np.array(LFPs_df), samplerate, t_start, 'Signals')

cmap = cm.Spectral
values = np.linspace(0, 1, np.array(LFPs_df).shape[1])
colormap = [to_hex(rgb) for rgb in cmap(values)]
for idx in range(np.array(LFPs_df).shape[1]): 
    view1.by_channel_params[f'ch{idx}', 'color'] = colormap[idx]

view1.params['display_labels'] = True
view1.params['scale_mode'] = 'same_for_all'
view1.auto_scale()
view1.params['xsize'] = winlen
win.add_view(view1)
win.navigation_toolbar.spinbox_xsize.setValue(winlen)

#Run
win.show()
app.exec()


### Flip omnetics headstage

Run the cell bellow if you believe that the omnetic headstage was reverted for this mouse during this recording (if you run it twice it will go back to the initial configuration).

In [ ]:
rec_ch_list_ID = LFPs_df.columns-ID*32
rec_ch_list_mouse = [value for value in rec_ch_list_ID if 0 <= value <= 31]
i = np.argmax(rec_ch_list_ID>=0)
inverted_chs = np.concatenate([range(16,32,1), range(0,16,1)], axis=0)
LFPs_df_mouse=LFPs_df.iloc[:,i:i+len(rec_ch_list_mouse)]
flipped_ch=(inverted_chs[LFPs_df_mouse.columns-(ID*32)])+(ID*32)
LFPs_df.columns.values[i:i+len(rec_ch_list_mouse)] = flipped_ch
LFPs_df = LFPs_df.sort_index(axis=1)

# Reselect electrodes
all_LFPcoordinates = all_LFPcoordinates.astype(str)
for region in all_LFPcoordinates.loc[mouse].index:
    locals()[region] = []
    locals()[f'{region}_ch'] = []
    
RecordedArea = []
ChoosenChannels = []
combined = []
if mouse:
    rec_ch_list_mouse = [value for value in rec_ch_list if 0+(ID*32) <= value <= 31+(ID*32)]
    for rec_ch in rec_ch_list_mouse:
        for idx, LFPcoord_str in enumerate(all_LFPcoordinates.loc[mouse]):
            region = all_LFPcoordinates.loc[mouse].index[idx]
            if LFPcoord_str != 'nan':
                LFPcoord = LFPcoord_str.split('_')[:2] # only take into account the 2 first of electrode of that region 
                num_ch = np.where(str(rec_ch-(ID*32)) == np.array(LFPcoord))[0]
                if len(num_ch) > 0:
                    region = all_LFPcoordinates.loc[mouse].index[idx]
                    LFP = locals()[region]
                    LFP = LFP-np.array(LFPs_df[(rec_ch)]) if len(LFP) > 0 else np.array(LFPs_df[(rec_ch)])
                    locals()[region] = LFP
                    locals()[f'{region}_ch'].append(rec_ch)
                    break
                continue
    
    for region in all_LFPcoordinates.loc[mouse].index:
        LFP = locals()[region]
        LFP_ch = locals()[f'{region}_ch']
        if len(LFP) > 0:
            combined = zscore(LFP[:,np.newaxis]) if len(combined) == 0 else np.append(combined, zscore(LFP[:,np.newaxis]), axis=1)
            RecordedArea.append(region) 
            ChoosenChannels.append(LFP_ch) 
else:
    print("/!/ No mouse name found in the path OR in the csv file '_LFP_coordinates_of_all_mice.csv'")
    mouse = '' # fill mouse name
    RecordedArea = ['EMG','S1','PFC','CA1'] 
    EMG = LFPs_df[0]
    S1 = LFPs_df[0]-LFPs_df[1]
    combined = np.stack([zscore(EMG), zscore(S1)], axis=1)

print(mouse)
print(RecordedArea)
print(ChoosenChannels) 

### Visualize detected oscillations 

Upload detected oscillation

In [ ]:
# SWR     
SWSdetection=folder_base / f'SWR_detection.csv'
#SWSdetection=folder_base / f'SWRproperties.csv'
SWR_prop = pd.read_csv(SWSdetection, index_col=0)  
#SWR_prop['toKeep'] = SWR_prop['toKeep'].astype(str)
#SWR_prop  = SWR_prop[SWR_prop['toKeep'].isin(['VRAI', 'True'])]   
SWR_start = np.transpose((SWR_prop['start time']).astype(int))
SWR_end = np.transpose((SWR_prop['end time']).astype(int))

# Spindles
Spdldetection=folder_base / f'SpindlesS1&PFC_detection.csv'
#Spdldetection=folder_base / f'Spindleproperties_S1&PFC.csv'
Spdl_prop = pd.read_csv(Spdldetection, index_col=0)  
#Spdl_prop['toKeep'] = Spdl_prop['toKeep'].astype(str)
#Spdl_prop  = Spdl_prop[Spdl_prop['toKeep'].isin(['VRAI', 'True'])]   

S1Spdl_prop= Spdl_prop[Spdl_prop['CTX']=='S1']
S1spdl_start = np.transpose(S1Spdl_prop['start time'].astype(int))
S1spdl_end = np.transpose(S1Spdl_prop['end time'].astype(int))

PFCSpdl_prop= Spdl_prop[Spdl_prop['CTX']=='PFC']
PFCspdl_start = np.transpose(PFCSpdl_prop['start time'].astype(int))
PFCspdl_end = np.transpose(PFCSpdl_prop['end time'].astype(int))

GlobalSpdl_prop= Spdl_prop[Spdl_prop['CTX']=='S1PFC']
GBLspdl_start = np.transpose(GlobalSpdl_prop['start time'].astype(int))
GBLspdl_end = np.transpose(GlobalSpdl_prop['end time'].astype(int))

print(f'{len(SWR_end)} SWR in CA1, {len(S1spdl_end)} Spdl in S1, {len(PFCspdl_end)} Spdl in PFC, {len(GBLspdl_end)} Global Spdl')

In [ ]:
app = mkQApp()
labels = ['AW_1', 'QW_2', 'NREM_3', 'IS_4',  'REM_5', 'undefined_6'] # Active wake, Quiet wake, NREM sleep, Intermediate sleep, REM sleep, undefined states
epoch_dur = 5 # define epoch duration in sec
winlen = 10 # default window length in sec

try: 
    scatter_indexes = {0: PFCspdl_start, 1: PFCspdl_end, 2: S1spdl_start, 3: S1spdl_end, 4: GBLspdl_start, 5: GBLspdl_end, 6: SWR_start, 7: SWR_end}
    scatter_channels = {0: [2], 1: [2], 2: [1], 3: [1], 4: [1,2], 5: [1,2], 6: [3], 7: [3]}
except: 
    scatter_indexes={}
    scatter_channels={}

#Create the main window that can contain several viewers
win = MainViewer(debug=False, show_auto_scale=True)
t_start=0.
#create a viewer for signal
source = AnalogSignalSourceWithScatter(combined, samplerate, t_start, scatter_indexes, scatter_channels, scatter_colors= {0: '#FFFFFF', 1: '#222222', 2: '#FFFFFF', 3: '#222222', 4: '#BBBBBB', 5: '#555555', 6: '#FFFFFF', 7: '#222222'}, channel_names=RecordedArea)
view1 = TraceViewer(source=source)

view1.params['xsize']= winlen
view1.params['display_labels'] = True
view1.params['scale_mode'] = 'same_for_all'
colormap = np.insert(['#88FF88', '#8888FF', '#FF8888']* 10, 0, '#FFFFFF')
for idx, ch in enumerate(RecordedArea): 
    view1.by_channel_params[f'ch{idx}', 'color'] = colormap[idx] #FF0000 red, #00FF00 green, and #0000FF blue
view1.auto_scale()


# FFT
view2 = TimeFreqViewer(source=source, name='FFT')

view2.params['show_axis'] = True
view2.params['timefreq', 'f_start'] = 1
view2.params['timefreq', 'f_stop'] = 200
view2.params['timefreq', 'deltafreq'] = 1 #interval in Hz
view2.params['xsize'] = winlen
for idx, ch in enumerate(RecordedArea):
    if ch == 'EMG': 
        view2.by_channel_params[f'ch{idx}', 'visible'] = False
    else:        
        view2.by_channel_params[f'ch{idx}', 'clim'] = 1
        view2.by_channel_params[f'ch{idx}', 'visible'] = True


# Scoring 
SleepScoring_filename = f'{dpath}/Sleep_Scoring_{len(labels)}Stages_{epoch_dur}sEpoch.csv'
source_epoch = CsvEpochSource(SleepScoring_filename, labels)
view3 = EpochEncoder(source=source_epoch, name='Sleep Scoring')

view3.params['xsize'] = winlen
view3.params['new_epoch_step'] = epoch_dur
view3.by_label_params['label0', 'color'] = '#ffcd69' #AW
view3.by_label_params['label1', 'color'] = '#69ffe6' #QW
view3.by_label_params['label2', 'color'] = '#69cfff' #NREM
view3.by_label_params['label3', 'color'] = '#cd69ff' #IS
view3.by_label_params['label4', 'color'] = '#ff69cd' #REM
view3.by_label_params['label5', 'color'] = '#8f8f8f' #undefined
view3.controls.hide()

#Run
win.add_view(view1)
win.add_view(view2)
win.add_view(view3)
win.navigation_toolbar.spinbox_xsize.setValue(winlen)
win.show()

app.exec()


### Extract example traces

In [ ]:
beg = 0
fin = beg + 800

Hypnogram

In [ ]:
labels = ['AW_1', 'QW_2', 'NREM_3', 'IS_4',  'REM_5', 'undefined_6'] # Active wake, Quiet wake, NREM sleep, Intermediate sleep, REM sleep, undefined states
epoch_dur=5
SleepScoring_filename = f'{dpath}/Sleep_Scoring_{len(labels)}Stages_{epoch_dur}sEpoch_{mouse}.csv'
SleepScoring_filename = f'{dpath}/Sleep_Scoring_{len(labels)}Stages_{epoch_dur}sEpoch.csv'
ScoringFile  = pd.read_csv(SleepScoring_filename)

# Upscale Scoring
ScoringFile['label']= ScoringFile['label'].str.extract(r'(\d+)', expand=False)
SleepScoredTS=np.array(ScoringFile['label'])

In [ ]:
# Plot Hynogram Total
colors = ['black', 'grey','darkgrey', 'lightgrey', 'white', 'red']
cmap = mcolors.ListedColormap(colors)
bounds = [1, 2, 3, 4, 5, 6, 7]
norm = mcolors.BoundaryNorm(boundaries=bounds, ncolors=len(colors))

# Extract and prepare DataFrame
df = pd.DataFrame(SleepScoredTS, columns=['Value'])
df['Value'] = df['Value'].astype(int)  # ensure values are ints for fmt="d"

# Plot
plt.figure(figsize=(8, 0.25))
heatmap = sns.heatmap(df['Value'].values.reshape(1, -1), cmap=cmap, norm=norm, fmt="d")

# Customize appearance
cbar = heatmap.collections[0].colorbar
cbar.remove()
plt.xticks([])
plt.yticks([])
plt.savefig(f'C:/Users/Manip2/Documents/Manuscripts/Figures_PNASreviews/FigureSupp_revised/Extract{mouse}_{folder_base.parts[-6]}_Hypno_total.png', format='png', bbox_inches='tight', pad_inches=0, transparent=True)
plt.show()

In [ ]:
# Plot Hynogram from beg to fin
colors = ['black', 'grey','darkgrey', 'lightgrey', 'white', 'red']
cmap = mcolors.ListedColormap(colors)
bounds = [1, 2, 3, 4, 5, 6, 7]
norm = mcolors.BoundaryNorm(boundaries=bounds, ncolors=len(colors))

# Extract and prepare DataFrame
df = pd.DataFrame(SleepScoredTS[int(beg/epoch_dur):int(fin/epoch_dur)], columns=['Value'])
df['Value'] = df['Value'].astype(int)  # ensure values are ints for fmt="d"

# Plot
plt.figure(figsize=(8, 0.25))
heatmap = sns.heatmap(df['Value'].values.reshape(1, -1), cmap=cmap, norm=norm, fmt="d")

# Customize appearance
cbar = heatmap.collections[0].colorbar
cbar.remove()
plt.xticks([])
plt.yticks([])
#plt.savefig(f'C:/Users/Manip2/Documents/Manuscripts/Figures_PNASreviews/FigureSupp_revised/Extract{mouse}_{folder_base.parts[-6]}_Hypno_{beg}to{fin}s.png', format='png', bbox_inches='tight', pad_inches=0, transparent=True)
plt.savefig(f'C:/Users/Manip2/Documents/Manuscripts/Figures_PNASreviews/FigureSupp_revised/Extract{mouse}_{folder_base.parts[-2]}_Hypno_{beg}to{fin}s.png', format='png', bbox_inches='tight', pad_inches=0, transparent=True)
plt.show()

Downsample LFPs

In [ ]:
S1extract=(S1[round(beg*samplerate):round(fin*samplerate)])
PFCextract=(PFC[round(beg*samplerate):round(fin*samplerate)])
CA1extract=(CA1[round(beg*samplerate):round(fin*samplerate)])
EMGextract=(EMG[round(beg*samplerate):round(fin*samplerate)])

if samplerate == 25000: 
    S1extractDS=S1extract[::250]
    PFCextractDS=PFCextract[::250]
    CA1extractDS=CA1extract[::250]
    EMGextractDS=EMGextract[::250]
else: 
    S1extractDS=S1extract[::10]
    PFCextractDS=PFCextract[::10]
    CA1extractDS=CA1extract[::10]
    EMGextractDS=EMGextract[::10]

In [ ]:
# All raw and filtered LFPs and EMG
plt.close()
plt.figure(figsize=(6, 2))
blend_factor = .5

hex_color = '#2E8B57'
rgb_color = mcolors.hex2color(hex_color)
lighter_color = (1 - blend_factor) * np.array(rgb_color) + blend_factor * np.array([1, 1, 1])
lighter_color_hex = mcolors.to_hex(lighter_color)

plt.plot(zscore(PFCextractDS)/2, color='seagreen', linewidth=.5)
plt.text(-10, np.mean(zscore(PFCextractDS)/2)+.5, 'raw PFC', color='seagreen', 
        ha='right', va='center', fontsize=6)  

hex_color = '#663399'
rgb_color = mcolors.hex2color(hex_color)
lighter_color = (1 - blend_factor) * np.array(rgb_color) + blend_factor * np.array([1, 1, 1])
lighter_color_hex = mcolors.to_hex(lighter_color)

plt.plot(zscore(S1extractDS)/2-3, color='rebeccapurple', linewidth=.5)
plt.text(-10, np.mean(zscore(S1extractDS)/2-3)+.5, 'raw S1', color='rebeccapurple', 
        ha='right', va='center', fontsize=6)  



hex_color = '#4169E1'
rgb_color = mcolors.hex2color(hex_color)
lighter_color = (1 - blend_factor) * np.array(rgb_color) + blend_factor * np.array([1, 1, 1])
lighter_color_hex = mcolors.to_hex(lighter_color)

plt.plot(zscore(CA1extractDS)/2.5-6, color='royalblue', linewidth=.5)
plt.text(-10, np.mean(zscore(CA1extractDS)/2.5-6)+.5, 'raw CA1', color='royalblue', 
        ha='right', va='center', fontsize=6)  

plt.plot(zscore(EMGextractDS)/5-9, color='black', linewidth=.5)
plt.axis('off')  # Hides the axis lines and labels
plt.text(-10, np.mean(zscore(EMGextractDS)/5-9), 'EMG', color='black', 
        ha='right', va='center', fontsize=6)  

#plt.savefig(f'C:/Users/Manip2/Documents/Manuscripts/Figures_PNASreviews/FigureSupp_revised/Extract{mouse}_{folder_base.parts[-6]}_AllLFPs_{beg}to{fin}s.svg', format='svg', bbox_inches='tight', pad_inches=0, transparent=True)
plt.savefig(f'C:/Users/Manip2/Documents/Manuscripts/Figures_PNASreviews/FigureSupp_revised/Extract{mouse}_{folder_base.parts[-2]}_AllLFPs_{beg}to{fin}s.svg', format='svg', bbox_inches='tight', pad_inches=0, transparent=True)
plt.show()
